<h5 style="font-weight:bold;padding: 0 5%">Name: Ou Wang</h5>
<h5 style="font-weight:bold;padding: 0 5%">uID: xxxxxxxx</h5>
<h1 style="font-weight:bold;text-align:center;padding: 0 5%">Are the daily supplied of finished motor gasoline and the confirmed cases of covid-19 correlated in the US?</h1>

<p style="padding: 0 5%;text-indent: 3em;">CDC has been recommending social distancing and quarantine, and the most of the organizations/companies have been keeping their employees working from home. The intuition is that the daily supplied gasoline may decrease during the covid-19 pandemicin US. The line plot in Figure 1 indeed showed that the gasoline supply started decreasing since 03/13/2020, when many companies started the working from home policies (from > 8,000 thousand barrels per day before 03/13/2020 to around 5,000 thousand barrels per day on 04/20/2020).</p>

<p style="padding: 0 5%;text-indent: 3em;">The barplot in Figure 1 indicated that the daily confirmed cases of covid-19 started showing a trend of decreasing between 04/24/2020 and 05/29/2020, which is one month after the gasoline supply started dropping. However, the Pearson correlation coefficient of -0.33 (Appendix e) did not suggesta strong correlation between gasoline supply and covid-19 cases. This could be due to that the changes of covid-19 cases and gasoline supply are not simultaneous. Also, the gas supply gradually increased back since 04/03/2020, and it reached to 8,000 thousand barrels per day again on 06/19/2020. Thus, the data points could be insufficient to show the correlation between these two variables.</p>

<p style="padding: 0 5%;text-indent: 3em;">Cumulative sum analysis was then performed to detect changes for gasoline supply and covid-19 cases (Figure 2 and 3). The results indicated that the gasoline supply decreased between 03/06/2020 and 04/24/2020, and the confirmed cases of covid-19 decreased between 05/11/2020 and 06/05/2020, suggestingthat reduced gasolinesupply may be associated with the reduced cases of covid-19. However, it may take about a month to show the effect. The reason could be that the reduced gasoline supply would lead to reduced human traveling, which decreases the chance of human contact and spreading covid-19.</p>

<p style="padding: 0 5%;text-indent: 3em;">In summary, although finished motor gasoline supply and confirmed cases of covid-19 did not show strong correlation, the cumulative sum analysis suggested that the daily cases of covid-19 indeed decreased, following the decease of motor gasoline supply. In addition, as shown in Figure 1, sincethe motor gasoline supply increased back to > 8,000 thousand barrels per day on 06/19/2020, the daily cases of covid-19 started skyrocketing. Therefore, we should follow CDC’s recommendationto stay at home to help flatten the curve. </p>

<h1 style="font-weight:bold;"> Figues </h1>

In [50]:
show(p)

In [67]:
show(p_case)

In [66]:
show(p_gas)

<h1 style="font-weight:bold;">Appendix</h1>

<h3 style="font-weight:bold;"> 1. Tools used </h3>

    Google BigQuery 

    Python

 
<h3 style="font-weight:bold;">2. Data Source </h3>

    US daily supplied of finished motor gasoline: US Energy Information administration, https://www.eia.gov/ 

    US daily covid-19 cases: Google BigQuery public dataset, bigquery-public-data.covid19_nyt.us_states
    
    
<h3 style="font-weight:bold;">3. Analysis methods </h3>

    (1) Pearson correlation was performed using corr() function in BigQuery. The data were scaled and centered before correlation analysis.  

    (2) Cumulative sum analysis: Cumulative Sum was calculated using the following formula: 

        St = min(⁡0, St−1+xt−𝜇−C) ∗ −1;

        Decrease detected if St > T.

        X: value of a variable (daily gasoline supply or covid-19 cases); 

        µ: moving average of a variable (daily gasoline supply or covid-19 cases); the dataset is first sorted by date, and µ is calculated for a window including current row and 3 preceding rows, due to large variation among covid-19 daily cases.  

        C: µ * 10%, to control sensitivity of change detection 

        T: threshold, the calculated St is compared with T, which is 2 times of the standard deviation for a window including current row and 3 preceding rows, due to large variation among covid-19 daily cases. Thus, the T is dynamic, instead of a constant. 

        When St (converted to positive number as shown in the formula) is greater than T, a decrease is detected.  

<h3 style="font-weight:bold;">4. SQL queries </h3>

In [2]:
# !export GOOGLE_APPLICATION_CREDENTIALS="/home/owang/.google/BigQuery.json"
%load_ext google.cloud.bigquery

<h3 style="color:blue;">a) Create dim_date table </h3>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.dim_date
    as (
        select 
            cast(format_date("%E4Y%m%d", dates) as numeric) as `DateKey`
            , format_date("%D", dates) as `Date`
            , extract(year from dates) as `Year`
            , extract(month from dates) as `Month`
            , extract(quarter from dates) as `Quarter`
            , extract(dayofweek from dates) as `DayOfWeek`
            , extract(isoweek from dates) as `ISOWeek`
            , format_date("%A", dates) as `DayName`
            , format_date("%B", dates) as `MonthName`
            , if(extract(dayofweek from dates)=1 or extract(dayofweek from dates)=7, 1, 0) as `IsWeekend`
        from unnest(generate_date_array('2010-01-01', '2030-01-01')) as `dates`
);

In [3]:
%%bigquery --project bigquery-281204 --use_rest_api
select *
from is6850.dim_date
limit 5
;

,DateKey,Date,Year,Month,Quarter,DayOfWeek,ISOWeek,DayName,MonthName,IsWeekend
0,20100224,02/24/10,2010,2,1,4,8,Wednesday,February,0
1,20100727,07/27/10,2010,7,3,3,30,Tuesday,July,0
2,20100821,08/21/10,2010,8,3,7,33,Saturday,August,1
3,20100830,08/30/10,2010,8,3,2,35,Monday,August,0
4,20100914,09/14/10,2010,9,3,3,37,Tuesday,September,0


<h3 style="color:blue;">b) Create covid-19 summary table, including daily cases/death and accumulative cases/death </h3>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.covid19_summary as
    with temp as (
      select date
        , sum(confirmed_cases) as cumulative_confirmed_cases
        , sum(deaths) as cumulative_deaths
      from `bigquery-public-data.covid19_nyt.us_states`
      group by date
      order by date
    )
    select date
     , cumulative_confirmed_cases
     , cumulative_confirmed_cases - first_value(cumulative_confirmed_cases) over (diff) as daily_confirmed_cases
     , cumulative_deaths
     , cumulative_deaths - first_value(cumulative_deaths) over (diff) as daily_death
    from temp
    window diff as (
      order by date
      rows between 1 preceding and current row
    );

In [4]:
%%bigquery --project bigquery-281204 --use_rest_api
select *
from is6850.covid19_summary
order by date
limit 5
;

,date,cumulative_confirmed_cases,daily_confirmed_cases,cumulative_deaths,daily_death
0,2020-01-21,1,0,0,0
1,2020-01-22,1,0,0,0
2,2020-01-23,1,0,0,0
3,2020-01-24,2,1,0,0
4,2020-01-25,3,1,0,0


<h3 style="color:blue;">c) Create summary table include gas consumption and daily covid-19 cases, saved in array to preserve order </h3>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.summary_daily_cases as
  with temp as (
    select 
        date
        , daily_confirmed_cases
        , Weekly_US_Product_Supplied_of_Finished_Motor_Gasoline_Thousand_Barrels_per_Day as gas
    from bigquery-281204.is6850.covid19_summary dc
    join bigquery-281204.is6850.gas_consumption gc
    on dc.date = gc.week_of
  )
  select 
    array_agg(
      struct(date, daily_confirmed_cases, gas)
      order by date) 
      as summary
  from temp
;

In [5]:
%%bigquery --project bigquery-281204 --use_rest_api
SELECT date
    , daily_confirmed_cases
    , gas 
FROM 
    `bigquery-281204.is6850.summary_daily_cases`,
    unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
limit 5
;

,date,daily_confirmed_cases,gas
0,2020-01-24,1,8793
1,2020-01-31,1,8933
2,2020-02-07,0,8722
3,2020-02-14,0,8918
4,2020-02-21,3,9035


<h3 style="color:blue;">d) Get standard devidation and mean then normalize data </h3>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.correlation_data as
with temp as (
  SELECT 
    date
    , daily_confirmed_cases
    , stddev(daily_confirmed_cases)
      over(statistics) as cases_stddev
    , avg(daily_confirmed_cases) 
      over(statistics) as cases_avg
    , gas
    , stddev(gas)
      over(statistics) as gas_stddev
    , avg(gas)
      over(statistics) as gas_avg
  FROM `bigquery-281204.is6850.summary_daily_cases`
  , unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
  window statistics as (
      order by date
      rows between unbounded preceding and unbounded following
  )
)
select
  date,
  (daily_confirmed_cases - cases_avg)/cases_stddev as norm_cases
  , (gas - gas_avg)/gas_stddev as norm_gas
from temp
;

<h3 style="color:blue;">e) Get Pearson Correlation coefficientcreate from above normalized data</h3>

In [6]:
%%bigquery --project bigquery-281204 --use_rest_api
SELECT round(corr(norm_cases, norm_gas), 4) as pearson
FROM `bigquery-281204.is6850.correlation_data`
;

,pearson
0,-0.3252


<h3 style="color:blue;">f) Prepare data for cumulative sum calculation</h3>

In [18]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table is6850.cusum_data as
with temp as (
SELECT 
  date
  , daily_confirmed_cases
  , avg(daily_confirmed_cases)
    over(moving_avg) as cases_c
  , stddev(daily_confirmed_cases)
    over(std_window) as case_t
  , gas
  , avg(gas)
    over(moving_avg) as gas_c
  , stddev(gas)
    over(std_window) as gas_t
FROM `bigquery-281204.is6850.summary_daily_cases`
, unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
window moving_avg as (
    order by date
    rows between 3 preceding and current row
    ), 
    std_window as (
      order by date
      rows between 3 preceding and current row
      )
)
select 
  * replace(
    cases_c*1.1 as cases_c
    , gas_c*1.1 as gas_c
    , case_t*2 as case_t
    , gas_t*2 as gas_t
    )
from temp
where date > "2020-03-01"
;

""


In [7]:
%%bigquery --project bigquery-281204 --use_rest_api
select *
from is6850.cusum_data
limit 5
;

,date,daily_confirmed_cases,cases_c,case_t,gas,gas_c,gas_t
0,2020-06-26,45528,35904.550,17735.875545,8561,9058.225,808.995056
1,2020-04-03,32318,15351.325,28227.940975,5065,8320.675,4199.996627
2,2020-06-12,25492,28199.050,4324.318983,7870,8407.300,609.386577
3,2020-06-19,30835,30135.050,5805.946492,8608,8779.925,893.407522
4,2020-03-27,17330,6486.700,16046.584268,6659,9526.275,2764.331565


<h3 style="color:blue;">g) Calculate cumulative sum</3>

In [21]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table is6850.cusum_table as
with temp as(
  SELECT *
  FROM `bigquery-281204.is6850.cusum_data`
  order by date
), 
temp2 as(
  select 
    date
    , case_t, 
    daily_confirmed_cases - cases_c as sum_cases, 
    gas_t, gas - gas_c as sum_gas
  from temp
)
select *
  , if(sum(sum_cases) over(cusum_win) > 0, 0, sum(sum_cases) over(cusum_win)) as cusum_case
  , if(sum(sum_gas) over(cusum_win) > 0, 0, sum(sum_gas) over(cusum_win)) as cusum_gas
from temp2
where date > "2020-03-01"
window cusum_win as (
  order by date
  rows between 1 preceding and current row
)
;

""


In [8]:
%%bigquery --project bigquery-281204 --use_rest_api
select * 
from is6850.cusum_table
limit 5
;

,date,case_t,sum_cases,gas_t,sum_gas,cusum_case,cusum_gas
0,2020-04-03,28227.940975,16966.675,4199.996627,-3255.675,0.000,-6122.950
1,2020-06-12,4324.318983,-2707.050,609.386577,-537.300,-2403.975,-747.600
2,2020-05-29,3455.894674,-3564.700,656.139213,-423.250,-10473.450,-899.125
3,2020-05-01,4316.360929,-3345.575,1407.605058,362.100,-3465.850,0.000
4,2020-03-20,5426.222596,3896.675,735.867742,-1384.200,0.000,-1963.850


<h3 style="font-weight:bold;">5. Python code - data visulization </h3>

<h3 style="color:blue;">a) Figure 1 daily covid-19 case vs gas consumption</3> 

In [ ]:
%%bigquery df_cases_gas --project bigquery-281204 --use_rest_api
SELECT date
    , daily_confirmed_cases
    , gas 
FROM 
    `bigquery-281204.is6850.summary_daily_cases`,
    unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
;

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d, Title
import numpy as np

df_cases_gas['date'] = df_cases_gas.date.astype('str')

output_notebook()

TOOLTIPS = [
    ("Date", "@date"),
    ("Confirmed Cases", "@daily_confirmed_cases"),
    ("Supplied Gas", "@gas")
]

p = figure(plot_width=1200, 
           plot_height=900,
           x_range=df_cases_gas.date,
           y_range=(0, 120000),
           tooltips=TOOLTIPS
          )

p.vbar(source=df_cases_gas,
       x='date',
       top='daily_confirmed_cases',
       width=0.5, 
       bottom=0,
       color='firebrick',
       legend_label="Daily Confirmed Covid-19 Cases"
      )

p.extra_y_ranges = {'gas_y': Range1d(0, 11000)}

p.line(df_cases_gas.date,
       df_cases_gas.gas,
       line_width=2,
       color='#73ccf0',
       y_range_name='gas_y',
       legend_label="Daily Supplied of Finished Motor Gasoline"
      )

p.circle(source=df_cases_gas,
         x='date',
         y='gas',
         color='#73ccf0',
         size=10,
         y_range_name='gas_y',
         legend_label="Daily Supplied of Finished Motor Gasoline"
        )

p.add_layout(LinearAxis(y_range_name='gas_y'), 'right')

p.xaxis.major_label_orientation = np.pi/4
p.xaxis[0].axis_label = 'Date'
p.yaxis[0].axis_label = 'US Daily Supplied of Finished Motor Gasoline (Thousand Barrels)'
p.yaxis[1].axis_label = 'US Daily Confirmed Covid-19 Cases'

p.add_layout(Title(text="Fig 1. Daily Confirmed Covid-19 Cases and Supplied of Finished Motor Gasoline", 
                   text_font_size="15pt",
                   align="center"
                  ), 
             'below')
p.add_layout(Title(text="-Hover Over the Points and Bars to View data", 
                   text_font_style="italic",
                   text_font_size="12pt",
                   align="center"
                  ), 
             'below')
p.add_layout(Title(text="Y-axis on the left is for US Product Supplied of Finished Motor Gasoline (in thousand barrels per day). Y-axis on the right is for the daily confirmed cases of Covid-19 in US.",
                   text_font_size="9pt",
                   align="center"
                  ), 
             'below')

<h3 style="color:blue;">b) Figure 2 cusum for covid-19 cases</3> 

In [ ]:
%%bigquery df_cusum --project bigquery-281204 --use_rest_api
select * replace(cusum_case*(-1) as cusum_case
                 , cusum_gas*(-1) as cusum_gas)
from is6850.cusum_table
order by date
;

In [61]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d, Title
import numpy as np

output_notebook()

def plot_cusum(df: 'pandas dataframe with 3 columns, date, threshold, cunsum',
              y_min: 'minimal y vlaue',
              y_max: 'maximal y value',
              title: 'title of the chart',
              pre_fix: 'prefix to title',
              post_fix: 'postfix to title and legend'):
    
    df['date'] = df.date.astype('str')
    
    p = figure(plot_width=900,
                    plot_height=600,
                    x_range=df.date,
                    y_range=(y_min, y_max),
                    title_location="below",
                   )
    
    p.line(source = df,
           x = 'date',
           y = df.columns[1],
           line_width=2,
           color='#f04511',
           legend_label=f'Threshold({post_fix})'
          )
    
    p.line(source = df,
           x = 'date',
           y = df.columns[2],
           line_width=2,
           color='#73ccf0',
           legend_label=f'Cusum({post_fix})'
          )
    
    p.add_layout(Title(text=f'{pre_fix} Cumulative Sum Analysis for Change Detection for {title}',
                       text_font_size="12pt",
                       align="center"
                      ),
                 'below')
    
    p.xaxis.major_label_orientation = np.pi/4
    
    return p

p_case = plot_cusum(df_cusum[['date', 'case_t', 'cusum_case']].copy(),
                    0,
                    40000,
                    'Daily Confirmed Covid-19 Cases in The US',
                    'Fig 2.',
                    'Cases'
                   )

Loading BokehJS ...

<h3 style="color:blue;">c) Figure 3 cusum gas consumption</3> 

In [64]:
p_gas = plot_cusum(df_cusum[['date', 'gas_t', 'cusum_gas']].copy(),
                    0,
                    8000,
                    'Daily Supplied of Finished Motor Gasoline in The US',
                    'Fig 3.',
                    'Gas'
                   )